In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories


In [4]:
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [6]:

import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir ="artifacts/prepare_callbacks/tensorboard_log_dir"+f"tb_logs_at_{timestamp}"
    
        return tf.keras.callbacks.TensorBoard(log_dir=Path(tb_running_log_dir))

    @property #A property can be accessed without () brackets
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= "artifacts/prepare_callbacks/checkpoint_dir/model.h5",
            save_best_only=True   #saves only best weights
           
        )

    def get_tb_ckpt_callbacks(self):  
        return [
            self._create_tb_callbacks,  #A property can be accessed without () brackets
            self._create_ckpt_callbacks
        ]

In [42]:
!pip install scipy

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,  #rescale my pictures
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],#(row,colimn,channel) channel is not needed
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation", #name of subset
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            #for every epoch it provides different version of same image
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,#rotate image by 40 radians
                horizontal_flip=True,
                width_shift_range=0.2,  #increase the width by 20%
                height_shift_range=0.2,   ##increase the height by 20%
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    #@staticmethod
    #def save_model(path: Path, model: tf.keras.Model):
     #   model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        '''self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )'''

        self.model.save(self.config.trained_model_path)

In [8]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()  #train,validation data generators
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 692/1250 [===============>..............] - ETA: 28:58 - loss: 8.2193 - accuracy: 0.6402

c:\Users\Bharadwaj\DeepCNNClassifier\env\lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - ETA: 0s - loss: 6.3998 - accuracy: 0.6856

c:\Users\Bharadwaj\DeepCNNClassifier\env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 4936s 4s/step - loss: 6.3998 - accuracy: 0.6856 - val_loss: 6.6269 - val_accuracy: 0.6755


In [ ]:
 1./255

0.00392156862745098

In [ ]:
def ex(x,**kwargs):
    print(locals())

In [ ]:
ex(x=3,y=5,z=7)

{'x': 3, 'kwargs': {'y': 5, 'z': 7}}


In [ ]:
e = dict(y=4,z=7)
ex(x=3,**e)

{'x': 3, 'kwargs': {'y': 4, 'z': 7}}
